# Syed 

# COSC 631 Database Information Systems II

# Query Optimizer

# Generation of a General Query Tree

In [2]:
import operator
import itertools
import sqlparse
import re
import tkinter as tk
from anytree import Node, RenderTree 
from pprint import pprint

# Creating an query execution plan for non-optimized queries

In [3]:
def generate_non_optimized_query_execution_plan(sql_statement): 
    #parse through the SQL statement
    parsed_sql = sqlparse.parse(sql_statement)[0]
    
    #define a list of keywords in the order of their execution
    keyword_order = ['SELECT', 'FROM', 'WHERE']
    
    #initialize a list to hold the SQL operations
    operations = {}
    
    #Split the SQL statement into tokens 
    tokens = sql_statement.split()
    
    
#     #iterate over the tokens in the parsed SQL
#     for item in parsed_sql.tokens:
#         # if the token is a keyword, add it to the operations list
#         if item.ttype is not None and item.value.upper() in keyword_order:
#             operation_type = item.value.strip()
#             operations[operation_type] = Node(operation_type)
            
#             #Set the parent of the operation node to the previous operation
#             if len(operations) > 1:
#                 prev_operation_type = list(operations.keys())[-2]
#                 operations[operation_type].parent = operations[prev_operation_type]
#     #iterate over the tokens again to add the remaining operations to the dictionary
#     for item in parsed_sql.tokens:
#         if item.ttype is not None and item.value.upper() not in keyword_order:
#             node_value = item.value.strip()
#             node_type = item.ttype[0]
#             node = Node(f"{node_value} ({node_type})", parent=operations[list(operations.keys())[-1]])
    
    
    #Iterate over the tokens in the parsed SQL
    for item in tokens:
        #If the token is a keyword, add it operations dictionary
        if item.upper() in keyword_order:
            operation_type = item.strip()
            operations[operation_type] = Node(operation_type)
            
            #Set the parent of the operation node to the previous operation
            if len(operations) > 1:
                prev_operation_type = list(operations.keys())[-2]
                operations[operation_type].parent = operations[prev_operation_type]
        
        #if the token is not a keyword, add it as a child of the last operation
        else:
            node_value = item.strip()
            node_type = "Name"
            node = Node(f"{node_value} ({node_type})", parent=operations[list(operations.keys())[-1]])
            
    #return the tree
    return RenderTree(operations['SELECT'])

# Example 1

In [4]:
sql_statement = "SELECT EMPLOYEE.EMPNO, POSITION FROM EMPLOYEE E, JOBHISTORY J WHERE E.EMPNO = J.EMPNO AND STARTDATE <= ENDDATE AND SALARY <= 3000"
tree = generate_non_optimized_query_execution_plan(sql_statement)
for pre, fill, node in tree:
    print(f"{pre}{node.name}")

SELECT
├── EMPLOYEE.EMPNO, (Name)
├── POSITION (Name)
└── FROM
    ├── EMPLOYEE (Name)
    ├── E, (Name)
    ├── JOBHISTORY (Name)
    ├── J (Name)
    └── WHERE
        ├── E.EMPNO (Name)
        ├── = (Name)
        ├── J.EMPNO (Name)
        ├── AND (Name)
        ├── STARTDATE (Name)
        ├── <= (Name)
        ├── ENDDATE (Name)
        ├── AND (Name)
        ├── SALARY (Name)
        ├── <= (Name)
        └── 3000 (Name)


# Example 2

In [5]:
sql_statement = '''SELECT LName, FName, DOB FROM Employee WHERE LName = "Aquarius" and City = NYC'''
tree = generate_non_optimized_query_execution_plan(sql_statement)
for pre, fill, node in tree:
    print(f"{pre}{node.name}")

SELECT
├── LName, (Name)
├── FName, (Name)
├── DOB (Name)
└── FROM
    ├── Employee (Name)
    └── WHERE
        ├── LName (Name)
        ├── = (Name)
        ├── "Aquarius" (Name)
        ├── and (Name)
        ├── City (Name)
        ├── = (Name)
        └── NYC (Name)


# Now optimize and parse queries

In [6]:
def optimize_and_parse_query(query):
    # Remove the STARTDATE <= ENDDATE condition from the WHERE clause
    parsed_query = sqlparse.parse(query)[0]
    where_clause = None
    for token in parsed_query.tokens:
        if isinstance(token, sqlparse.sql.Where):
            where_clause = token
            break

    if where_clause:
        where_str = str(where_clause)
        where_str = where_str.replace("AND STARTDATE <= ENDDATE", "")
        where_clause = sqlparse.parse(where_str)[0]
        parsed_query.tokens = [t for t in parsed_query.tokens if not isinstance(t, sqlparse.sql.Where)]
        if where_clause:
            parsed_query.tokens.insert(-1, where_clause)

    select_clause = None
    for token in parsed_query.tokens:
        if isinstance(token, sqlparse.sql.TokenList):
            for subtoken in token.tokens:
                if isinstance(subtoken, sqlparse.sql.Identifier):
                    if subtoken.value.upper() == 'DOB':
                        token.tokens.remove(subtoken)
                        break
            select_clause = token
            break

    # Parse the optimized query and create a dictionary to store the query tree
    query_tree = {'type': parsed_query.get_type(), 'value': str(parsed_query)}

    # Add the SELECT clause to the tree
    if select_clause:
        query_tree['select'] = [subtoken.value for subtoken in select_clause.tokens if isinstance(subtoken, sqlparse.sql.Identifier)]

    # Add the FROM clause to the tree
    for token in parsed_query.tokens:
        if isinstance(token, sqlparse.sql.IdentifierList):
            query_tree['from '] = [subtoken.value for subtoken in token.tokens if isinstance(subtoken, sqlparse.sql.Identifier)]

    # Add the WHERE clause to the tree
    for token in parsed_query.tokens:
        if isinstance(token, sqlparse.sql.Where):
            query_tree['where '] = []
            for where_token in token.tokens:
                if isinstance(where_token, sqlparse.sql.Comparison):
                    query_tree['where '].append({'left': where_token.left.value, 'operator': where_token.normalized, 'right': where_token.right.value})

    # Return the query tree
    return query_tree


# Example 1:

In [7]:
query = "SELECT EMPLOYEE.EMPNO, POSITION FROM EMPLOYEE E, JOBHISTORY J WHERE E.EMPNO = J.EMPNO AND STARTDATE <= ENDDATE AND SALARY <= 3000"
optimized_query = optimize_and_parse_query(query)
pprint(optimized_query)

{'from ': ['EMPLOYEE E', 'JOBHISTORY J'],
 'select': ['EMPLOYEE.EMPNO'],
 'type': 'SELECT',
 'value': 'SELECT EMPLOYEE.EMPNO, POSITION FROM EMPLOYEE E, JOBHISTORY JWHERE '
          'E.EMPNO = J.EMPNO  AND SALARY <= 3000 '}


# Example 2: 

In [16]:
query = '''SELECT LName, FName, DOB FROM Employee WHERE LName = "Aquarius" and City = "NYC"'''
optimized_query = optimize_and_parse_query(query)
pprint(optimized_query)

{'from ': ['LName', 'FName'],
 'select': ['LName', 'FName'],
 'type': 'SELECT',
 'value': 'SELECT LName, FName,  FROM EmployeeWHERE LName = "Aquarius" and '
          'City = "NYC" '}


# Example 3:

In [9]:
query = "SELECT first_name, last_name FROM employees WHERE department = 'Sales' AND salary > 50000 AND STARTDATE <= ENDDATE"
optimized_query = optimize_and_parse_query(query)
pprint(optimized_query)

{'from ': ['first_name', 'last_name'],
 'select': ['first_name', 'last_name'],
 'type': 'SELECT',
 'value': 'SELECT first_name, last_name FROM employeesWHERE department = '
          "'Sales' AND salary > 50000  "}


In [17]:
def generate_optimized_query_execution_plan(sql_statement): 
    #parse through the SQL statement
    parsed_sql = sqlparse.parse(sql_statement)[0]
    
    #define a list of keywords in the order of their execution
    keyword_order = ['SELECT', 'FROM', 'WHERE']
    
    #initialize a list to hold the SQL operations
    operations = {}
    
    #Split the SQL statement into tokens 
    tokens = sql_statement.split()
    
    
#     #iterate over the tokens in the parsed SQL
#     for item in parsed_sql.tokens:
#         # if the token is a keyword, add it to the operations list
#         if item.ttype is not None and item.value.upper() in keyword_order:
#             operation_type = item.value.strip()
#             operations[operation_type] = Node(operation_type)
            
#             #Set the parent of the operation node to the previous operation
#             if len(operations) > 1:
#                 prev_operation_type = list(operations.keys())[-2]
#                 operations[operation_type].parent = operations[prev_operation_type]
#     #iterate over the tokens again to add the remaining operations to the dictionary
#     for item in parsed_sql.tokens:
#         if item.ttype is not None and item.value.upper() not in keyword_order:
#             node_value = item.value.strip()
#             node_type = item.ttype[0]
#             node = Node(f"{node_value} ({node_type})", parent=operations[list(operations.keys())[-1]])
    
    
    #Iterate over the tokens in the parsed SQL
    for item in tokens:
        #If the token is a keyword, add it operations dictionary
        if item.upper() in keyword_order:
            operation_type = item.strip()
            operations[operation_type] = Node(operation_type)
            
            #Set the parent of the operation node to the previous operation
            if len(operations) > 1:
                prev_operation_type = list(operations.keys())[-2]
                operations[operation_type].parent = operations[prev_operation_type]
        
        #if the token is not a keyword, add it as a child of the last operation
        else:
            node_value = item.strip()
            node_type = "Name"
            node = Node(f"{node_value} ({node_type})", parent=operations[list(operations.keys())[-1]])
            
    #return the tree
    return RenderTree(operations['SELECT'])

# Example 1: Optimized Execution Plan

In [18]:
optimized_query = "SELECT EMPLOYEE.EMPNO, POSITION FROM EMPLOYEE E, JOBHISTORY J WHERE E.EMPNO = J.EMPNO AND SALARY <= 3000"
tree = generate_optimized_query_execution_plan(optimized_query)
for pre, fill, node in tree:
    print(f"{pre}{node.name}")

SELECT
├── EMPLOYEE.EMPNO, (Name)
├── POSITION (Name)
└── FROM
    ├── EMPLOYEE (Name)
    ├── E, (Name)
    ├── JOBHISTORY (Name)
    ├── J (Name)
    └── WHERE
        ├── E.EMPNO (Name)
        ├── = (Name)
        ├── J.EMPNO (Name)
        ├── AND (Name)
        ├── SALARY (Name)
        ├── <= (Name)
        └── 3000 (Name)


# Example 2: Optimized Execution Plan

In [19]:
sql_statement = '''SELECT LName, FName FROM Employee WHERE LName = "Aquarius"'''
tree = generate_optimized_query_execution_plan(sql_statement)
for pre, fill, node in tree:
    print(f"{pre}{node.name}")

SELECT
├── LName, (Name)
├── FName (Name)
└── FROM
    ├── Employee (Name)
    └── WHERE
        ├── LName (Name)
        ├── = (Name)
        └── "Aquarius" (Name)


# Example 3: Optimized Execution Plan

In [20]:
sql_statement = "SELECT first_name, last_name FROM employees WHERE department = 'Sales' AND salary > 50000"
tree = generate_optimized_query_execution_plan(sql_statement)
for pre, fill, node in tree:
    print(f"{pre}{node.name}")

SELECT
├── first_name, (Name)
├── last_name (Name)
└── FROM
    ├── employees (Name)
    └── WHERE
        ├── department (Name)
        ├── = (Name)
        ├── 'Sales' (Name)
        ├── AND (Name)
        ├── salary (Name)
        ├── > (Name)
        └── 50000 (Name)


# Now building the Graphical User Interface And a Better Optimized Query Function

In [21]:
def optimize_query(query):
    # Parse the query into a parse tree using sqlparse
    parsed = sqlparse.parse(query)[0]
    
    # Get the WHERE clause
    where_clause = None
    for token in parsed.tokens:
        if isinstance(token, sqlparse.sql.Where):
            where_clause = token
            break

    # Remove the STARTDATE <= ENDDATE condition from the WHERE clause
    if where_clause:
        where_str = str(where_clause)
        where_str = where_str.replace("AND STARTDATE <= ENDDATE", "")
        where_clause = sqlparse.parse(where_str)[0]
    
    # Remove "DOB" from the SELECT clause
    select_clause = None
    for token in parsed.tokens:
        if isinstance(token, sqlparse.sql.TokenList):
            for subtoken in token.tokens:
                if isinstance(subtoken, sqlparse.sql.Identifier):
                    if subtoken.value.upper() == 'DOB':
                        token.tokens.remove(subtoken)
                        break
            select_clause = token
            break
    
    # Remove all city conditions from the WHERE clause
#     if where_clause:
#         where_tokens = where_clause.tokens
#         new_tokens = []
#         skip_next = False
#         for i, token in enumerate(where_tokens):
#             if skip_next:
#                 skip_next = False
#                 continue
#             if str(token).lower() == 'city':
#                 j = i + 2
#                 while j < len(where_tokens) and str(where_tokens[j]).lower() != 'and':
#                     j += 1
#                 skip_next = True
#             else:
#                 new_tokens.append(token)
#         where_clause = sqlparse.sql.Where(new_tokens)
 # Remove all city conditions from the WHERE clause
    if where_clause:
        where_str = str(where_clause)
        where_str = re.sub(r"and City = [A-Za-z\s]+", "", where_str)
        where_clause = sqlparse.parse(where_str)[0]

        
    
    # Recreate the optimized query
    parsed.tokens = [t for t in parsed.tokens if not isinstance(t, sqlparse.sql.Where)]
    if where_clause:
        parsed.tokens.insert(-1, where_clause)
    optimized_query = str(parsed)
    return optimized_query


In [22]:
def on_optimize_query_click():
    #Get the input query from the text box
    query = input_text.get('1.0', 'end-1c')
    
    #Optimize the query using the optimize_query function
    optimized_query = optimize_query(query)
    optimized_query_tree = generate_optimized_query_execution_plan(optimized_query)
    
    # Display the optimized query in the output text box
    output_text.delete('1.0', tk.END)
    output_text.insert('1.0', optimized_query)

    # Display the query execution plan in the second output text box
    output_text_tree.delete('1.0', tk.END)
    output_text_tree.insert('1.0', optimized_query_tree)
    
#Create the main window
root = tk.Tk()
root.title('SQL Statement Query Optimizer')

# Create the input text box
input_text = tk.Text(root, height = 10, width = 70)
input_text.pack(side = tk.TOP, padx = 10, pady = 10)

# Create the output text box for optimized query
output_text = tk.Text(root, height = 10, width = 150)
output_text.pack(side = tk.TOP, padx = 10, pady = 10)

# Create the output text box for query tree
output_text_tree = tk.Text(root, height = 10, width = 150)
output_text_tree.pack(side = tk.TOP, padx = 10, pady = 10)

#create the optimize query button
optimize_button = tk.Button(root, text = 'Optimize Query', command = on_optimize_query_click)
optimize_button.pack(side = tk.TOP, padx = 10, pady = 10)

#Start the main event loop
root.mainloop()
